In [1]:
import tkinter as tk 
import tkinter.filedialog as fd
import pandas as pd
import warnings
from threading import Thread
import datetime
import re
import numpy as np

warnings.filterwarnings("ignore")

root = tk.Tk()

def open_looker_button():
    
    def open_looker_file():
        filenames = fd.askopenfilenames()
        if filenames:
            dir_name = fd.askdirectory()
            if dir_name:
                tk.messagebox.showinfo("File loaded","Press OK to start processing. This might take a minute.")
                df = pd.DataFrame()
                for filename in filenames:
                    df_new = pd.read_excel(filename,dtype='object',na_filter = False)
                    df = pd.concat([df,df_new],ignore_index=True)
                if 'Subdomain' in df.columns:
                    source = 'collect_'
                else:
                    source = 'rms_'

                save_file = dir_name + '/' + source + 'Booklist_' + str(datetime.date.today()) + '.xlsx'
                df = list_builder(df,save_file)    
                tk.messagebox.showinfo("Your file is ready!", "Your file was saved as: "+save_file+".\n You can exit or select more files to process")
            else:
                tk.messagebox.showinfo("Process cancelled","No directory selected")
        else:
            tk.messagebox.showinfo("Process cancelled","No file selected")
    open_looker_file()
    
def open_fmf_button():
    def fmf_mm():
        filename = fd.askopenfilename()
        if filename:            
            dir_name = fd.askdirectory()            
            if dir_name:
                tk.messagebox.showinfo("File loaded","Press OK to start processing. This might take a minute.")
                df = pd.read_excel(filename,dtype='object',na_filter = False)
                save_file = dir_name + '/fmf_mm' + str(datetime.date.today()) + '.xlsx'
                df = match(df,save_file)
                tk.messagebox.showinfo("Your file is ready!", "Your file was saved as: "+save_file+".\n You can exit or select more files to process")
            else:
                tk.messagebox.showinfo("Process cancelled","No directory selected")
        else:
            tk.messagebox.showinfo("Process cancelled","No file selected")
    fmf_mm()

def list_builder(df,save_file):
    
    if 'Subdomain' in df.columns:
        coll = df
        coll = coll.iloc[:,1:]
        coll = coll.drop_duplicates(subset=['Isbn'], keep='first')
        coll['keep'] = 'no'
        regex = r'[1-9]'
        coll.replace('', 'unknown', inplace=True)
        for index, row in coll.iterrows():
            isbn = row['Isbn']
            if re.search(regex, isbn):
                coll.at[index,'keep'] = 'yes'

        coll = coll.drop(coll[coll['keep'] == 'no'].index)
        coll = coll.drop(columns=['keep'])
        coll = coll.rename(columns={
            'Subdomain':'Section number',
            'Isbn':'Identifiers (VBID, ISBN or Product SKU - Required)',
            'Authors':'Author full name'})

        template_columns =[
            'Author first name',
            'Author last name',
            'Publication date (mm/yyyy)',
            'Language',
            'Format',
            'Duration',
            'Course code',
            'Enrollment',
            'Content deadline',
            'Launch date',
            'Price',
            'Currency',
            'Courseware',
            'Wants IA Price'
        ]

        column_index = 0
        while column_index < len(template_columns):
            coll[template_columns[column_index]] = ''
            column_index += 1
        coll_cols = list(coll.columns)
        coll_cols.insert(12, coll_cols.pop(coll_cols.index('Section number')))
        coll_cols.insert(0, coll_cols.pop(coll_cols.index('Identifiers (VBID, ISBN or Product SKU - Required)')))
        coll_cols.insert(1, coll_cols.pop(coll_cols.index('Publisher')))
        coll_cols.insert(2, coll_cols.pop(coll_cols.index('Title')))
        coll_cols.insert(3, coll_cols.pop(coll_cols.index('Edition')))
        coll = coll[coll_cols]
        today = datetime.datetime.today()
        template_date = today + datetime.timedelta(days=30)
        coll['Course code'] = 'coll'
        coll['Content deadline'] = template_date.strftime('%m/%d/%y')
        coll['Launch date'] = template_date.strftime('%m/%d/%y')
        coll['Enrollment'] = '25'
        coll["Content deadline"] = pd.to_datetime(coll["Content deadline"]).dt.normalize()
        coll["Launch date"] = pd.to_datetime(coll["Launch date"]).dt.normalize()
        coll["Content deadline"] = coll["Content deadline"].astype(str)
        coll["Launch date"] = coll["Launch date"].astype(str)
        template = pd.read_excel('resources/Booklist_SearchTemplate.xlsx')
        coll_upload = pd.DataFrame(np.concatenate([template, coll.values]), columns=template.columns)        
        coll_upload.to_excel(save_file, engine='xlsxwriter', index=False) 
    else:
        rms = df 
        rms = rms.drop_duplicates(subset=['Isbn'], keep='first') 
        rms = rms.iloc[:,1:]
        rms = rms.drop_duplicates(subset=['Isbn'], keep='first')
        rms['keep'] = 'no'
        regex = r'[1-9]'
        for index, row in rms.iterrows():
            isbn = row['Isbn']
            if re.search(regex, isbn):
                rms.at[index,'keep'] = 'yes'
            if row['Est. Sales (RMS)'] == '':
                rms.at[index,'Est. Sales (RMS)'] = 25
            elif row['Est. Sales (RMS)'] < 1:
                rms.at[index,'Est. Sales (RMS)'] = 25
        rms.replace('', 'unknown', inplace=True)
        rms = rms.drop(rms[rms['keep'] == 'no'].index)
        rms = rms.drop(columns=['keep'])
        rms = rms.rename(columns={
            'Organization Name':'Section number',
            'Isbn':'Identifiers (VBID, ISBN or Product SKU - Required)',
            'Author':'Author full name'})
        template_columns =[
            'Author first name',
            'Author last name',
            'Publication date (mm/yyyy)',
            'Language',
            'Format',
            'Duration',
            'Course code',
            'Enrollment',
            'Content deadline',
            'Launch date',
            'Price',
            'Currency',
            'Courseware',
            'Wants IA Price'
        ]
        column_index = 0
        while column_index < len(template_columns):
            rms[template_columns[column_index]] = ''
            column_index += 1
        rms = rms.drop(columns=['Enrollment'])
        rms = rms.rename(columns={'Est. Sales (RMS)':'Enrollment'})
        rms_cols = list(rms.columns)
        rms_cols.insert(13, rms_cols.pop(rms_cols.index('Section number')))
        rms_cols.insert(0, rms_cols.pop(rms_cols.index('Identifiers (VBID, ISBN or Product SKU - Required)')))
        rms_cols.insert(1, rms_cols.pop(rms_cols.index('Publisher')))
        rms_cols.insert(2, rms_cols.pop(rms_cols.index('Title')))
        rms_cols.insert(3, rms_cols.pop(rms_cols.index('Edition')))
        rms_cols.insert(4, rms_cols.pop(rms_cols.index('Author full name')))
        rms_cols.insert(13, rms_cols.pop(rms_cols.index('Enrollment')))
        rms = rms[rms_cols]
        today = datetime.datetime.today()
        template_date = today + datetime.timedelta(days=30)
        rms['Course code'] = 'rms'
        rms['Content deadline'] = template_date.strftime('%m/%d/%y')
        rms['Launch date'] = template_date.strftime('%m/%d/%y')
        rms["Content deadline"] = pd.to_datetime(rms["Content deadline"]).dt.normalize()
        rms["Launch date"] = pd.to_datetime(rms["Launch date"]).dt.normalize()
        rms["Content deadline"] = rms["Content deadline"].astype(str) 
        rms["Launch date"] = rms["Launch date"].astype(str)
        template = pd.read_excel('resources/Booklist_SearchTemplate.xlsx')
        rms_upload = pd.DataFrame(np.concatenate([template, rms.values]), columns=template.columns)        
        rms_upload.to_excel(save_file, engine='xlsxwriter', index=False)

def match(df,save_file):
    #open rights request export

    

    # morning match
    # df2 US approvals 
    # filters to instituition NetSuite or Akademos, Institution country US, and dist set Reseller US (USD)
    df2 = df.loc[(df["Institution"].str.contains("NetSuite") | df["Institution"].str.contains( "Akademos")) &
                    (df["Institution's Country"] == "US") &
                    (df['Already In'].str.contains(r'Reseller\s-\sUS\s\(USD\)') & df['Already In'].notnull())]

    # df3 CA approvals
    # filters to instituition NetSuite or Bookware, Institution country CA, and dist set Reseller CA (CAD)
    df3 = df.loc[(df["Institution"].str.contains("NetSuite") | df["Institution"].str.contains( "Bookware")) &
                    (df["Institution's Country"] == "CA") &
                    (df['Already In'].str.contains(r'Reseller\s-\sCA\s\(CAD\)') & df['Already In'].notnull())]

    # df4 US denials
    # filters to instituition NetSuite, Institution country US, and dist set Exclusion set: unavailable to US resellers
    df4 = df.loc[(df["Institution"].str.contains("NetSuite")  &
                    (df["Institution's Country"] == "US") &
                    (df['Already In'].str.contains(r'Exclusion\s+set:\s+unavailable\s+to\s+US\s+resellers') & df['Already In'].notnull()))]

    # df5 CA denials
    # filters to instituition NetSuite or Bookware, Institution country CA, and dist set Exclusion set: unavailable to CA resellers
    df5 = df.loc[(df["Institution"].str.contains("NetSuite") | df["Institution"].str.contains( "Bookware")) &
                    (df["Institution's Country"] == "CA") &
                    (df['Already In'].str.contains(r'Exclusion\s+set:\s+unavailable\s+to\s+CA\s+resellers') & df['Already In'].notnull())]

    # add status and denial reason code for morning match
    df2 = df2.assign(**{'Change Status To': 'Approved'}) #.assign is used add the values to the  column
    df3 = df3.assign(**{'Change Status To': 'Approved'})
    df4 = df4.assign(**{'Change Status To': 'Denied'})
    df4 = df4.assign(**{'Denial Reason Code': '5'})
    df5 = df5.assign(**{'Change Status To': 'Denied'})
    df5 = df5.assign(**{'Denial Reason Code': '5'})
    morning_match = pd.concat([df2, df3, df4,df5], ignore_index=True)   #.concat is used to combine the dataframes
    matches_to_remove = morning_match.assign(**{'Change Status To': ''}) 
    matches_to_remove = matches_to_remove.assign(**{'Denial Reason Code': ''})

    find_my_friends = df[~df.isin(matches_to_remove)].dropna(how='all') #.isin is used to remove the rows that are in matches_to_remove

    #find my friends
    # separate requests by status
    nm_oh = find_my_friends.loc[(df['Request Status'] == 'NEEDS_MATCH')|(df['Request Status'] =='ON_HOLD')]
    nm_oh = nm_oh[['VBID','Request Status','Most Recent Comment']]
    new = find_my_friends.loc[(df['Request Status'] == 'NEW')]
    new = new[['VBID','Request Status','Most Recent Comment']]
    other = find_my_friends.loc[(df['Request Status'] == 'PROCESSING')|(df['Request Status'] == 'WAITING_LEGAL')|(df['Request Status'] == 'WAITING_PUBLISHER')|(df['Request Status'] == 'WAITING_INSTITUTION')]
    other = other[['VBID','Request Status','Most Recent Comment']]



    new_nm = pd.merge(new,nm_oh,how='outer',on='VBID',indicator=True)
    new_nm = new_nm.loc[new_nm['_merge'] == 'both']
    new_nm = new_nm.drop(columns=['Request Status_y','Most Recent Comment_y','_merge'])
    new_nm = new_nm.rename(columns={'Request Status_x':'Change Status To','Most Recent Comment_x':'Comment'})

    other_nm = pd.merge(other,nm_oh,how='outer',on='VBID',indicator=True)
    other_nm = other_nm.loc[other_nm['_merge'] == 'both']
    other_nm = other_nm.drop(columns=['Request Status_y','Most Recent Comment_y','_merge'])
    other_nm = other_nm.rename(columns={'Request Status_x':'Change Status To','Most Recent Comment_x':'Comment'})

    others = other.drop(other[other['Request Status'] == 'WAITING_PUBLISHER'].index)

    new_other = pd.merge(new,others,how='outer',on='VBID',indicator=True)
    new_other = new_other.loc[new_other['_merge'] == 'both']
    new_other = new_other.drop(columns=['Request Status_y','Most Recent Comment_y','_merge'])
    new_other = new_other.rename(columns={'Request Status_x':'Change Status To','Most Recent Comment_x':'Comment'})

    friends = pd.concat([new_nm,other_nm,new_other],ignore_index=True)
    friends = friends.drop_duplicates(subset=['VBID'],keep='first')

    find_friends = pd.merge(friends,df,how='outer',on='VBID',indicator=True)
    find_friends = find_friends.loc[find_friends['_merge'] == 'both']
    find_friends = find_friends.drop(columns=['Change Status To_y','Comment_y','_merge'])
    find_friends = find_friends.rename(columns={'Change Status To_x':'Change Status To','Comment_x':'Comment'})
    cols = list(find_friends.columns)
    cols.insert(4, cols.pop(0))
    cols.insert(3, cols.pop(1))
    find_friends = find_friends[cols]

    mask = find_friends['Change Status To'] == find_friends['Request Status']
    find_friends = find_friends.loc[~mask]

    fmf_mm = pd.concat([morning_match,find_friends],ignore_index=True)

    # export to excel
    fmf_mm.to_excel(save_file, engine='xlsxwriter',index=False)





looker_button = tk.Button(root,text="Prepare Looker Booklist", command=lambda:Thread(target=open_looker_button).start())
looker_button.grid(row=0,column=0)
fmf_button = tk.Button(root,text="Find My Friends/Morning Match", command=lambda:Thread(target=open_fmf_button).start())
fmf_button.grid(row=1,column=0)
close_button = tk.Button(root,text="EXIT", command=lambda: root.destroy())
close_button.grid(row=2,column=0)




w = 250
h = 100
screen_width = root.winfo_screenwidth()  # Width of the screen
screen_height = root.winfo_screenheight() # Height of the screen
 
# Calculate Starting X and Y coordinates for Window
x = (screen_width/2) - (w/2)
y = (screen_height/2) - (h/2)
 
root.geometry('%dx%d+%d+%d' % (w, h, x, y))
root.grid_rowconfigure(0,weight=1)
root.grid_columnconfigure(0,weight=1)
root.configure(background='black')
root.title('Morning Routines')
root.mainloop()



    
